# B-Trees: `btree` module

**A demonstration notebook for the `btree` module**

This is a demonstration notebook for the first deliverable of the discipline **Algorithms Project II**, lectured by **Professor Reginaldo Cordeiro dos Santos Filho** at the **Federal University of Pará (UFPA)**.

## Goals

According to the deliverable specification document, the goals to be met are as follows:

1. The source-code shall be presented to the Professor.
2. The program shall build a B-Tree of degree m chosen by the Professor.
3. The group will be asked to insert some register keys into the B-Tree.
    - The program shall treat the attempt of inserting repeated keys.
4. The group will be asked to remove some register keys of the B-Tree.
    - The program shall treat the attempt of removing an inexistent key.
5. The group will be asked to search for some register keys in the B-Tree.
    - The result returned shall be: True (found) or False (not found).

## Requirements

In [1]:
from os import getcwd
from sys import path as sys_path
from random import shuffle

sys_path.append(getcwd())
from btree import *

## Example 1

This example was taken straight from the Professor's presentation slides about B-Trees. Throughout this example, the following sequence will be inserted into a `BTree` object:

50, 20, 30, 37, 42, 47, 41, 60, 31, 32, 43, 44, 61 and 62.

### Creating a `BTree` object of degree `2`

In [2]:
tree = BTree(2)

In [3]:
tree

[]

### Insertion Examples

#### Inserting `50`, `20`, `30` and `37`

In [4]:
tree.insert(50,20,30,37)

In [5]:
tree

[20, 30, 37, 50]

#### Inserting `42`

This insertion will cause the creation of a new page.

In [6]:
tree.insert(42)

In [7]:
tree

      [37]      
[20, 30] [42, 50]

#### Inserting `47` and `41`

In [8]:
tree.insert(47,41)

In [9]:
tree

          [37]          
[20, 30] [41, 42, 47, 50]

#### Inserting `60`

In [10]:
tree.insert(60)

In [11]:
tree

         [37, 47]         
[20, 30] [41, 42] [50, 60]

#### Inserting `31`, `32`, `43`, `44`, `61` and `62`

In [12]:
tree.insert([31,32,43,44,61,62])

In [13]:
tree

                     [37, 47]                     
[20, 30, 31, 32] [41, 42, 43, 44] [50, 60, 61, 62]

#### Inserting `33` and `45`

In [14]:
tree.insert(33)

In [15]:
tree

                   [31, 37, 47]                   
[20, 30] [32, 33] [41, 42, 43, 44] [50, 60, 61, 62]

In [16]:
tree.insert(45)

In [17]:
tree

                  [31, 37, 43, 47]                  
[20, 30] [32, 33] [41, 42] [44, 45] [50, 60, 61, 62]

#### Inserting `63`

In [18]:
tree.insert(63)

In [19]:
tree

                        [43]                        
            [31, 37]            [47, 61]            
[20, 30] [32, 33] [41, 42] [44, 45] [50, 60] [62, 63]

### Removal Examples

In [20]:
tree = BTree(2, [50, 20, 30, 37, 42, 47, 41, 60, 31, 32, 43, 44, 61, 62, 33, 45, 63, 38, 40])

In [21]:
tree

                            [43]                            
              [31, 37]              [47, 61]              
[20, 30] [32, 33] [38, 40, 41, 42] [44, 45] [50, 60] [62, 63]

#### Case 1: removing `38`

Since 38 is in a leaf page which has more than the minimum number of keys (which, in this case, is 2), the number can be removed without further treatment.

In [22]:
tree.remove(38)

In [23]:
tree

                          [43]                          
             [31, 37]             [47, 61]             
[20, 30] [32, 33] [40, 41, 42] [44, 45] [50, 60] [62, 63]

#### Case 2: removing `43`

Given that 43 is not in a leaf page (in fact, quite the opposite), the element has to be replaced by its predecessor or sucessor. The implementation of this B-Tree verifies both sides and obtains an element of the leaf page that has more elements – in this case, `43` will be replaced with `42`.

In [24]:
tree.remove(43)

In [25]:
tree

                        [42]                        
            [31, 37]            [47, 61]            
[20, 30] [32, 33] [40, 41] [44, 45] [50, 60] [62, 63]

#### Case 3: removing `33` instead of `43`

In a B-Tree, all pages must have a minimum amount of keys, which corresponds to the degree of the B-Tree. In these examples, the degree is 2, which means that the minimum amount of keys is 2 and the maximum is 4. Naturally, the root page is the only exception to this rule.

Therefore, when a page that has only the minimum amount of keys gets a key removed of itself, it violates the rule of the B-Tree, and measures must be taken in order to maintain the rule in force.

Before attempting to merge any pages (which is a more drastic, computationally expensive move), the B-Tree must verify the adjacent pages for the possibility of borrowing an element from a page that has more than the minimum amount of keys.

In this case, `33` is being removed from `Page` object `[ 32 | 33 ]`. Upon detection of rule violation, the `BTree` object will ascertain the possibility of borrowin an element. This possibility, of course, exists, since `Page` object `[ 40 | 41 | 42 ]` has an element to spare.

In [26]:
tree = BTree(2, [50, 20, 30, 37, 42, 47, 41, 60, 31, 32, 43, 44, 61, 62, 33, 45, 63, 40])

In [27]:
tree

                          [43]                          
             [31, 37]             [47, 61]             
[20, 30] [32, 33] [40, 41, 42] [44, 45] [50, 60] [62, 63]

In [28]:
tree.remove(33)

In [29]:
tree

                        [43]                        
            [31, 40]            [47, 61]            
[20, 30] [32, 37] [41, 42] [44, 45] [50, 60] [62, 63]

#### Case 4: removing `41`

In this case, since no `Page` object is able to borrow an element, the `Page` object has to be "demoted", or merged with an adjacent page (and taking the middle element of the parent page along).

This can unleash a chain-effect up to the root `Page` object, as it happens with this example.

In [30]:
tree.remove(41)

In [31]:
tree

                  [31, 43, 47, 61]                  
[20, 30] [32, 37, 40, 42] [44, 45] [50, 60] [62, 63]

## Example 2

This example was taken straight from the Professor's presentation slides about B-Trees – to be more specific, in the exercise section. Throughout this example, the following sequence will be inserted into a `BTree` object:

10, 20, 30, 40, 50, 3, 4, 11, 8, 9, 13, 25, 28, 17, 33, 36, 43, 45, 52, 55 and 48.

In [32]:
tree = BTree(2, [10, 20, 30, 40, 50, 3, 4, 11, 8, 9, 13, 25, 28, 17, 33, 36, 43, 45, 52, 55, 48])

In [33]:
tree

                              [30]                              
                [10, 20]                [40, 50]                
[3, 4, 8, 9] [11, 13, 17] [25, 28] [33, 36] [43, 45, 48] [52, 55]

### Partial B-Tree 1: remove `45`, `30` and `28`

In [34]:
tree.remove(45, 30, 28)

In [35]:
tree

                    [10, 25, 40, 50]                    
[3, 4, 8, 9] [11, 13, 17, 20] [33, 36] [43, 48] [52, 55]

### Partial B-Tree 2: remove `50`, `8`, `10`, `4`, `20`, `40`, `55`, `17`, `33`, `11` and `36`

In [36]:
tree.remove(50, 8, 10, 4, 20, 40, 55, 17, 33, 11, 36)

In [37]:
tree

         [43]         
[3, 9, 13, 25] [48, 52]

### Partial B-Tree 3: remove `3`, `9` and `52`

In [38]:
tree.remove(3,9,52)

In [39]:
tree

[13, 25, 43, 48]

## Testing

This testing code aims to exhaustively test the `BTree` and `Page` classes. It was extremely useful to catch bugs and areas for improvement throughout the code, and it is a powerful representation of the capabilities of this implementation.

It creates `BTree` objects of degree 1 to 10, adding a random sequence of 1000 numbers, finding another sequence of the same 1000 numbers and removing the same 1000 numbers in another order. During the removal, after the removal of each element, the remaining ones are searched for again, to make sure that no other element has been mistakenly removed of the `BTree` object. If any of these steps go wrong, an exception will be raised.

### Test settings

In [40]:
min_degree = 1
max_degree = 10
number_of_elements = 1000

In [41]:
for degree in range(min_degree, max_degree + 1):
    print("Current degree: {}...".format(degree))
    to_add = list(range(1, number_of_elements))
    to_find = list(range(1, number_of_elements))
    to_remove = list(range(1, number_of_elements))
    shuffle(to_add)
    shuffle(to_find)
    shuffle(to_remove)

    tree = BTree(degree, to_add)

    for i in to_find:
        in_tree, page_pointer, page_index = tree.find(i)
        if not in_tree:
            raise Exception("{} not found!".format(i))

    for i in range(len(to_remove)):
        if True:
            tree.remove(to_remove[i])
            for i in to_remove[i + 1 :]:
                in_tree, page_pointer, page_index = tree.find(i)
                if not in_tree:
                    raise Exception("{} not found!".format(i))
        else:
            break

Current degree: 1...
Current degree: 2...
Current degree: 3...
Current degree: 4...
Current degree: 5...
Current degree: 6...
Current degree: 7...
Current degree: 8...
Current degree: 9...
Current degree: 10...
